Problem Title: Maze

Accepted Languages:

- C++, Java, Python 2.7 and 3.4

Submission:

- Submit a single file containing your solution to the problem. It should output only the answer requested (NO DEBUG OUTPUT).

Grading:

20% - Code Quality
40% - Big O (efficiency)
40% - Correctness

Description:

Alright, there's not much to say. You're solving a maze.

Rules:

Each input file will begin with a number T, the number of test cases.

Each test case has two integers n and m on their own line. (1 <= n, m <= 5000). Then n lines follow, each with m characters. The characters are '.' if that space can be walked on and '#' otherwise. There are also two exceptions, 's' and 'e', in the grid which represent the starting point and desired end point. 's' and 'e' may also be walked on.

From any 'walkable' square, you may move up (U), down (D), left (L), or right (R) if that next square is also 'walkable'.

For each test case, print the shortest path from 's' to 'e' in the maze using the characters 'U', 'D', 'L', 'R' describing the path. If no path exists, then output ':('. If multiple shortest paths exist, print the path which would come first alphabetically.

Constraints:
1 <= n, m <= 5000

Input (read from standard in):

```
3
2 2
s.
.e
1 2
se
3 3
.#e
.#.
s#.
```

Output (write to standard out):

```
DR
R
:(
```

Explanation:

1) Both DR and RD work, but DR comes first in the alphabet.
2) The only path is to go right, so R.
3) There is no path, so have a sad face.


### Input Generators

In [42]:
input = """3
2 2
s.
.e
1 2
se
3 3
.#e
.#.
s#."""

def line_generator(input):
    """Simulates reading from standard input
    
    Every time you call next on it, you get the next line from stdin.
    
    """
    for line in input.split('\n'):
        yield line
        
def example_generator(line):
    """Read example from stdin and parse it into the appropriate data structure"""
    
    def maze_generator(n):
        for _ in range(n):
            row = next(line)
            
            yield list(row)
    
    while True:
        n, m = [int(num) for num in next(line).split()]
    
        yield list(maze_generator(n))

### Main Loop

In [43]:
from collections import namedtuple

Point = namedtuple('Point', ['x', 'y'])
Node = namedtuple('Node', ['path_len', 'loc'])



def find(char, maze):
    """Return Point containing the character
    
    Only use for finding 's' and 'e'
    
    """
    for i, row in enumerate(maze):
        for j, elem in enumerate(row):
            if elem == char:
                return Point(i, j)

def do_maze(maze):
    """Print the shortest path from the start node in the maze to the end node
    
    1. Find start and end nodes

    3. Run dijkstra's from start to end
    4. Print out shortest path
    
    """
    start, end = find('s', maze), find('e', maze)
            
    backpointers = dijkstra(maze, start, end)
    
    def trail_generator(backpointers, end):
        """Follow the trail of backpointers and report your trail"""

        x, y = end
        while backpointers[x][y] != 'N/A':
            direction = backpointers[x][y]
            yield direction
            
            x += inverse_directions[direction][0]
            y += inverse_directions[direction][1]
            
    path = list(trail_generator(backpointers, end))
    real_path = [mirror[direction] for direction in reversed(path)]
    
    print(':(' if not real_path else ''.join(real_path))

### Dijkstra's Algorithm

In [44]:
import copy

direction = {(1,0): 'D', (0,-1): 'L', (-1,0): 'U', (0,1): 'R'}
inverse_directions = {symbol: direction for direction, symbol in direction.items()}
mirror = {'D': 'U', 'U': 'D', 'L': 'R', 'R': 'L'}

class Node:
    def __init__(self, path_len, loc):
        self.path_len, self.loc = path_len, loc
    
    def __lt__(self, other):
        return self.path_len < other.path_len
    
    def __eq__(self, other):
        return self.loc == other.loc
    
    def __repr__(self):
        return '({},{}) -> {}'.format(self.loc.x, self.loc.y, self.path_len)
    
    def __hash__(self):
        return hash('({},{})'.format(*self.loc))

def dijkstra(maze, start, end):
    """Return the shortest path from start to end
    
    1. Make backpointer and shortest path matrices
    2. Make nodes heap for getting the next node to explore
    3. Run Dijkstras
    
    """
    n, m = len(maze), len(maze[0])
    
    backpointers, shortest_paths = copy.deepcopy(maze), copy.deepcopy(maze)
    for i, row in enumerate(maze):
        for j, elem in enumerate(row):
            backpointers[i][j], shortest_paths[i][j] = 'N/A', np.inf
            
    nodes = set()
    for i, row in enumerate(maze):
        for j, elem in enumerate(row):
            nodes.add(Node(np.inf, Point(i, j)))
    
    nodes.remove(Node(shortest_paths[start.x][start.y], start))
    nodes.add(Node(0, start)) # start point has a shortest path of zero to get us started
    shortest_paths[start.x][start.y] = 0
            
    while True:
        node = min(nodes)
        nodes.remove(node)
        
        if node.loc == end:
            break
        
        for dx, dy in zip([-1, 0, 1, 0], [0, 1, 0, -1]):
            x_, y_ = node.loc.x+dx, node.loc.y+dy
            
            if not 0 <= x_ < n or not 0 <= y_ < m:
                continue # don't consider points off the grid
                
            if maze[x_][y_] == '#':
                continue # don't consider walls
            
            if node.path_len+1 < shortest_paths[x_][y_]:
                # Update nodes set
                nodes.remove(Node(shortest_paths[x_][y_], Point(x_, y_)))
                nodes.add(Node(node.path_len+1, Point(x_, y_)))
                
                # Update shortest path and backpointer matrixes
                shortest_paths[x_][y_] = node.path_len+1
                backpointers[x_][y_] = mirror[direction[(dx, dy)]] # point back to this node
                
    return backpointers

### Input

In [46]:
line = line_generator(input)
example = example_generator(line)

T = next(line)

for _ in range(int(T)):
    maze = next(example)
    
    do_maze(maze)

RD
R
:(
